In [1]:
import pandas as pd
import numpy as np
#data visualization
import matplotlib.pyplot as plt
import librosa

from scipy.stats import kurtosis
from scipy.stats import skew

pd.options.display.precision = 10

## Umgebungsvariablen

In [8]:
feature_data_path = '''C:/studium/studium/CAS_PML/Projekt_Arbeit/earthquake/Daten/all/earthquakeFeatures_ext.csv'''

# Feature laden

In [9]:
train_data = pd.read_csv(feature_data_path, dtype={'acoustic_data': np.int16, 'time_to_failure': np.float64})

In [13]:
train_data.iloc[:,1:93].head()

,mean,std,kurt,skew,min,max,1%q,5%q,25%q,75%q,...,mfcc_kurt,mfcc_skew,mfcc_min,mfcc_max,mfcc_1%q,mfcc_cent_5%q,mfcc_25%q,mfcc_75%q,mfcc_95%q,mfcc_99%q
0,4.8841133333,5.1011061306,33.6624812935,-0.0240611666,-98.0,104.0,-8.0,-2.0,3.0,7.0,...,1.7926250934,1.1048759751,-92.1321830370,-26.1769160053,-90.0217627353,-86.1592492713,-79.7193050066,-65.7660047365,-50.0668287365,-32.7776445855
1,4.8460000000,5.3572687695,107.6590450983,0.3925487681,-154.0,181.0,-8.0,-2.0,3.0,7.0,...,8.0171198003,2.1118007117,-94.6178824145,8.7770193809,-91.3045186191,-86.4059601901,-80.3792653428,-66.6538662214,-49.0992852678,-22.1753170523
2,4.8111600000,5.3222446671,110.4614763269,0.4131461554,-154.0,181.0,-8.0,-2.0,2.0,7.0,...,8.4097814588,2.1560211118,-93.5586801788,8.4034865258,-90.8004274661,-87.0472882567,-80.2364598913,-66.9781009956,-49.1871208009,-21.8952768726
3,4.7970666667,5.4298018331,102.2839738481,0.3948502125,-154.0,181.0,-8.0,-2.0,2.0,7.0,...,7.6028397846,2.0502703394,-91.5577131484,5.1744835678,-89.8584868555,-86.7583032378,-79.7758637117,-65.7594193299,-46.1734484492,-25.5316359156
4,4.7963466667,6.2361343193,112.5711765807,0.4967878159,-154.0,181.0,-9.0,-2.0,2.0,7.0,...,8.7301748746,2.3213224679,-97.9336550547,14.4407606366,-93.0912226709,-85.1069205537,-79.4264194550,-65.0716703608,-44.3139940022,-8.9330467796


## Testdaten vorbereiten

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    features.iloc[:,1:93], features.iloc[:,-1], random_state=0, test_size=0.25)

# Create Model Workbench

## Imports

In [8]:
# Scaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

# Model selection
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures

# Modell
from sklearn.ensemble import GradientBoostingRegressor
from catboost import CatBoostRegressor, Pool
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

# GradientBoost

Pipe with preprocessing and one regressor

In [27]:
pipe = Pipeline([('preprocessing',PolynomialFeatures()), ('regressor', GradientBoostingRegressor())])

In [32]:
param_grid = [
    {'regressor': [GradientBoostingRegressor(n_estimators=1000, learning_rate=0.1, max_depth=1,loss='lad',random_state=0)],
     'regressor__loss':['lad','ls'],
     'regressor__max_depth':[1,np.abs2,3],
     'preprocessing': [None]}]

Find best parameter with grid search

In [34]:
grid = GridSearchCV(pipe, param_grid, scoring='neg_mean_absolute_error', cv=5, n_jobs=-1)
grid.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('preprocessing', PolynomialFeatures(degree=2, include_bias=True, interaction_only=False)), ('regressor', GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_...one, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid=[{'regressor': [GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
     ...se)], 'regressor__loss': ['lad', 'ls'], 'regressor__max_depth': [1, 2, 3], 'preprocessing': [None]}],
       pre_dispatch='2*n_jobs', refit=T

Print Result

In [35]:
print("Best params:\n{}\n".format(grid.best_params_))
print("Best cross-validation score: {:.2f}".format(grid.best_score_))
print("Test-set score: {:.2f}".format(grid.score(X_test, y_test)))
print("Test-set: mean absolute error : {:.2f}".format(mean_absolute_error(y_test, grid.predict(X_test))))

Best params:
{'preprocessing': None, 'regressor': GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=1000, n_iter_no_change=None, presort='auto',
             random_state=0, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False), 'regressor__loss': 'ls', 'regressor__max_depth': 3}

Best cross-validation score: -1.85
Test-set score: -1.82
Test-set: mean absolute error : 1.82


## Feature Importence

In [ ]:
(pd.Series(grid.best_estimator_.named_steps['regressor'].feature_importances_, index=X_train.columns)
   .nlargest(26)
   .plot(kind='barh',figsize=(15, 8))) 
#quelle: https://stackoverflow.com/questions/44511636/matplotlib-plot-feature-importance-with-feature-names

## Model selection
 - Random forrest
 - Other GradientBoost (AdaBoost)

 - Feed Forward NN (with Karas)
 - Reccurent Neuronal Net: LSTM (with Karas)
 - Convolutional NN https://www.kaggle.com/michael422/spectrogram-convolution https://www.kaggle.com/michael422/spectrogram-convolution
 
 
 - CatBoost vs. Light GBM vs. XGBoost https://www.kdnuggets.com/2018/03/catboost-vs-light-gbm-vs-xgboost.html
 - CatBoost-GridSearch https://setscholars.net/2019/02/19/how-to-find-optimal-parameters-for-catboost-using-gridsearchcv-for-regression-in-python/